In [2]:
!pip install numpy
!pip install ortools
!pip install onnx
!pip install torch
!pip install torchvision
!pip install pysmt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 57.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.4/294.4 kB 36.0 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-metadata 1.14.0 requires protobuf<4.21,>=3.20.3, but you have protobuf 4.25.0 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.7/15.7 MB 43.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.1/319.1 kB 7.2 MB/s eta 0:00:00


In [3]:
!pip install pynever

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 kB 1.3 MB/s eta 0:00:00


In [ ]:
!pip install --upgrade pip


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 10.3 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2


In [6]:
!pip install pypoman

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 159.3/159.3 kB 8.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for pycddlib
  Running setup.py clean for pycddlib
Failed to build pycddlib
ERROR: Could not build wheels for pycddlib, which is required to install pyproject.toml-based projects


In [7]:
import numpy as np
import matplotlib.pyplot as plt
import pypoman
import scipy.spatial

import pynever.strategies.abstraction as abst
import pynever.nodes as nodes


def plot_starset(p_starset: abst.StarSet):
    for p_star in p_starset.stars:

        alpha_vertexes = pypoman.compute_polytope_vertices(p_star.predicate_matrix, p_star.predicate_bias)
        x_vertexes = []
        for v in alpha_vertexes:
            v_ver = np.zeros((v.shape[0], 1))
            for k in range(v.shape[0]):
                v_ver[k, 0] = v[k]
            new_x_v = p_star.center + np.matmul(p_star.basis_matrix, v_ver)
            x_vertexes.append(new_x_v)

        x1_plot = []
        x2_plot = []
        for v in x_vertexes:
            x1_plot.append(v[0, 0])
            x2_plot.append(v[1, 0])

        for k in range(len(x1_plot)):
            for j in range(len(x1_plot)):
                plt.plot([x1_plot[k], x1_plot[j]], [x2_plot[k], x2_plot[j]])

        plt.scatter(x1_plot, x2_plot)

    plt.grid()
    plt.show()


def get_extreme_points(p_starset: abst.StarSet):
    x_vertexes = []
    for p_star in p_starset.stars:
        alpha_vertexes = pypoman.compute_polytope_vertices(p_star.predicate_matrix, p_star.predicate_bias)
        for v in alpha_vertexes:
            v_ver = np.zeros((v.shape[0], 1))
            for k in range(v.shape[0]):
                v_ver[k, 0] = v[k]
            new_x_v = p_star.center + np.matmul(p_star.basis_matrix, v_ver)
            new_x_v = np.array([new_x_v[0, 0], new_x_v[1, 0]])
            x_vertexes.append(new_x_v)

    x_vertexes = np.array(x_vertexes)
    return x_vertexes


def plot_convex_hull(points: np.ndarray):
    hull = scipy.spatial.ConvexHull(points)
    vertexes = points[hull.vertices, :]
    vertexes = np.vstack((vertexes, vertexes[0, :]))
    plt.plot(vertexes[:, 0], vertexes[:, 1])
    plt.grid()
    plt.show()


# Bidimensional Example with abstract verification

weight_matrix_1 = np.ones((2, 2))
weight_matrix_2 = np.ones((2, 2))
# weight_matrix_3 = np.ones((2, 2))
weight_matrix_3 = np.zeros((2, 2))

weight_matrix_1[1, 1] = -1
weight_matrix_2[1, 1] = -1
# weight_matrix_3[0, 1] = 0
weight_matrix_3[0, 0] = 1.0 / 5.0
weight_matrix_3[0, 1] = 1.0 / 5.0
weight_matrix_3[1, 0] = 2

bias_1 = np.zeros(2)
bias_2 = np.zeros(2)
bias_3 = [-0.4, -2]

net_names = ["FC_NET_1_1_1_1"]

neuron_relevance = True
refinement_percentage = 1

final_hulls = []
for i in range(len(net_names)):

    print("\n" + net_names[i] + "\n")

    abs_net = abst.AbsSeqNetwork(net_names[i])

    abs_net.add_node(
        abst.AbsFullyConnectedNode("AbsFC_1", nodes.FullyConnectedNode("FC_1", (2,), 2, weight_matrix_1, bias_1)))

    abs_net.add_node(abst.AbsReLUNode("AbsReLU_1", nodes.ReLUNode("ReLU_1", (2,)), "best_n_neurons", [10]))

    abs_net.add_node(
        abst.AbsFullyConnectedNode("AbsFC_2", nodes.FullyConnectedNode("FC_2", (2,), 2, weight_matrix_2, bias_2)))

    abs_net.add_node(abst.AbsReLUNode("AbsReLU_2", nodes.ReLUNode("ReLU_1", (2,)), "best_n_neurons", [10]))

    #abs_net.add_node(
    #    abst.AbsFullyConnectedNode("AbsFC_3", nodes.FullyConnectedNode("FC_3", 2, 2, weight_matrix_3, bias_3)))

    #abs_net.add_node(abst.AbsSigmoidNode("AbsSig_1", nodes.SigmoidNode("Sig_1", 2), approx_levels=2))

    C = np.zeros((4, 2))
    C[0, 0] = 1
    C[1, 0] = -1
    C[2, 1] = 1
    C[3, 1] = -1

    d = np.ones((4, 1))
    d[0, 0] = 1
    d[1, 0] = 1
    d[2, 0] = 1
    d[3, 0] = 1

    star = abst.Star(C, d)
    abs_input = abst.StarSet({star})
    current_node = abs_net.get_first_node()
    aux_points = get_extreme_points(abs_input)
    plot_convex_hull(aux_points)
    while current_node is not None:
        print(current_node)
        abs_input = current_node.forward(abs_input)
        print("Forward ended")
        plot_starset(abs_input)
        print(abs_input)
        for star in abs_input.stars:
            aux_points = get_extreme_points(star)
            plot_convex_hull(aux_points)
        current_node = abs_net.get_next_node(current_node)

ModuleNotFoundError: ignored

In [5]:
import numpy as np
import matplotlib.pyplot as plt
import pypoman
import scipy.spatial
import polytope
import torch

import pynever.strategies.abstraction as abst
import pynever.nodes as nodes
import pynever.strategies.verification as ver
import pynever.networks as networks
import pynever.utilities as utils
import pynever.strategies.conversion as conv


def plot_star(p_star: abst.Star, show=False, title=None, color = np.array([0.5, 0.5, 1])):

    #color = np.random.rand(3)
    alpha_vertexes = pypoman.compute_polytope_vertices(p_star.predicate_matrix, p_star.predicate_bias)
    x_vertexes = []
    for v in alpha_vertexes:
        v_ver = np.zeros((v.shape[0], 1))
        for k in range(v.shape[0]):
            v_ver[k, 0] = v[k]
        new_x_v = p_star.center + np.matmul(p_star.basis_matrix, v_ver)
        x_vertexes.append(new_x_v)

    x1_plot = []
    x2_plot = []
    for v in x_vertexes:
        x1_plot.append(v[0, 0])
        x2_plot.append(v[1, 0])

    for k in range(len(x1_plot)):
        for j in range(len(x1_plot)):
            plt.plot([x1_plot[k], x1_plot[j]], [x2_plot[k], x2_plot[j]], color=color)

    plt.scatter(x1_plot, x2_plot, color=color)
    if show:
        if title is not None:
            plt.title(title)
        plt.grid()


def plot_starset(p_starset: abst.StarSet, title, color = np.array([0.5, 0.5, 1])):
    for p_star in p_starset.stars:

        plot_star(p_star, color=color)

    if title is not None:
        plt.title(title)
    plt.grid()


def get_star_extreme_points(p_star: abst.Star):

    x_vertexes = []
    alpha_vertexes = pypoman.compute_polytope_vertices(p_star.predicate_matrix, p_star.predicate_bias)
    for v in alpha_vertexes:
        v_ver = np.zeros((v.shape[0], 1))
        for k in range(v.shape[0]):
            v_ver[k, 0] = v[k]
        new_x_v = p_star.center + np.matmul(p_star.basis_matrix, v_ver)
        new_x_v = np.array([new_x_v[0, 0], new_x_v[1, 0]])
        x_vertexes.append(new_x_v)

    x_vertexes = np.array(x_vertexes)

    return x_vertexes

def get_starset_extreme_points(p_starset: abst.StarSet):
    x_vertexes = []
    for p_star in p_starset.stars:
        alpha_vertexes = pypoman.compute_polytope_vertices(p_star.predicate_matrix, p_star.predicate_bias)
        for v in alpha_vertexes:
            v_ver = np.zeros((v.shape[0], 1))
            for k in range(v.shape[0]):
                v_ver[k, 0] = v[k]
            new_x_v = p_star.center + np.matmul(p_star.basis_matrix, v_ver)
            new_x_v = np.array([new_x_v[0, 0], new_x_v[1, 0]])
            x_vertexes.append(new_x_v)

    x_vertexes = np.array(x_vertexes)
    return x_vertexes


def plot_convex_hull(points: np.ndarray, title = None, color=None, alpha=1, label=None):

    hull = scipy.spatial.ConvexHull(points)
    vertexes = points[hull.vertices, :]
    vertexes = np.vstack((vertexes, vertexes[0, :]))
    if color is None:
        plt.fill(vertexes[:, 0], vertexes[:, 1], alpha=0.5, label=label)
    else:
        plt.fill(vertexes[:, 0], vertexes[:, 1], color=color, alpha=alpha, label=label)

    if title is not None:
        plt.title(title)
    plt.grid()


def print_star_data(p_star: abst.Star):

    print("PREDICATE CONSTRAINTS:")
    for row in range(p_star.predicate_matrix.shape[0]):
        constraint = ""
        for col in range(p_star.predicate_matrix.shape[1]):
            if p_star.predicate_matrix[row, col] < 0:
                sign = "-"
            else:
                sign = "+"
            constraint = constraint + f"{sign} {abs(p_star.predicate_matrix[row, col])} * x_{col} "

        constraint = constraint + f"<= {p_star.predicate_bias[row, 0]}"
        print(constraint)

    print("VARIABLES EQUATIONS:")
    for row in range(p_star.basis_matrix.shape[0]):
        equation = f"z_{row} = "
        for col in range(p_star.basis_matrix.shape[1]):
            if p_star.basis_matrix[row, col] < 0:
                sign = "-"
            else:
                sign = "+"
            equation = equation + f"{sign} {abs(p_star.basis_matrix[row, col])} * x_{col} "

        if p_star.center[row, 0] < 0:
            c_sign = "-"
        else:
            c_sign = "+"
        equation = equation + f"{c_sign} {abs(p_star.center[row, 0])}"
        print(equation)

ModuleNotFoundError: ignored

In [ ]:
overapprox = polytope.qhull(np.array([[0, 0], [-2, 0], [2, 2]]))

plot_convex_hull(overapprox.vertices, color='green', alpha=0.7, label="ReLU Over-Approximation", title="ReLU Over-Approximation")
plt.xlim([-3,3])
plt.ylim([-1,3])
plt.xlabel("$z_j$", size=15)
plt.ylabel("$y_j$", size=15)
plt.axhline(0, color='black')
plt.axvline(0, color='black')
plt.legend()
plt.savefig(f"ReLU Overapprox.pdf")
plt.show()